In [1]:
import pandas as pd
df = pd.read_excel("Bus Planning.xlsx")
df.head()

,start location,end location,start time,end time,activity,line,energy consumption,bus
0,ehvgar,ehvbst,05:03:00,05:07:00,material trip,NaN,1.9800,1
1,ehvbst,ehvapt,05:07:00,05:31:00,service trip,401.0,10.8036,1
2,ehvapt,ehvapt,05:31:00,06:04:00,idle,NaN,5.0000,1
3,ehvapt,ehvbst,06:04:00,06:29:00,service trip,401.0,10.8600,1
4,ehvbst,ehvbst,06:29:00,06:31:00,idle,NaN,5.0000,1


In [6]:
df2 = df.copy()
df2["start time"] = pd.to_datetime(df2["start time"], format="%H:%M:%S")
df2["end time"] = pd.to_datetime(df2["end time"], format="%H:%M:%S")

# Trek ze van elkaar af → dit geeft een timedelta
df2["duur"] = df2["end time"] - df2["start time"]

df2.head(5)

,start location,end location,start time,end time,activity,line,energy consumption,bus,duur
0,ehvgar,ehvbst,1900-01-01 05:03:00,1900-01-01 05:07:00,material trip,NaN,1.9800,1,0 days 00:04:00
1,ehvbst,ehvapt,1900-01-01 05:07:00,1900-01-01 05:31:00,service trip,401.0,10.8036,1,0 days 00:24:00
2,ehvapt,ehvapt,1900-01-01 05:31:00,1900-01-01 06:04:00,idle,NaN,5.0000,1,0 days 00:33:00
3,ehvapt,ehvbst,1900-01-01 06:04:00,1900-01-01 06:29:00,service trip,401.0,10.8600,1,0 days 00:25:00
4,ehvbst,ehvbst,1900-01-01 06:29:00,1900-01-01 06:31:00,idle,NaN,5.0000,1,0 days 00:02:00


In [49]:
def batterij(df, max_bat, percentage, gezondheid, min_percentage):
    """ df = dataframe, percentage = tot hoeveel procent je hem wil opladen, gezondheid = hoe goed de batterij is (dus tot hoeveel procent je hem ivm een nieuwe batterij kan opladen), min_percentage = min percentage dat de batterij mag zijn."""
    n = len(df)
    hvl_bus = len(df["bus"].unique())
    max_bat = int(max_bat)
    percentage = int(percentage)
    gezondheid = int(gezondheid)
    min_percentage = int(min_percentage)

    bat_status = max_bat * (gezondheid / 100) #
    bat_begin = bat_status * (percentage / 100) # 90 procent
    bat_min = bat_status * (min_percentage / 100)

    i = 0
    incorrect = 0

    for b in range(1, hvl_bus ):
        bat_moment = bat_begin
        while i <= n and df["bus"][i] == b:
            if df["activity"][i] == "idle":
                minutes = df["duur"][i].total_seconds() / 60
                energy_consumption = minutes * (df["energy consumption"][i] / 60)
                bat_moment -= energy_consumption
            else:
                bat_moment -= df["energy consumption"][i]

            if bat_moment < bat_min:
                incorrect += 1
                bat_percentage = (bat_moment / bat_status) * 100
                print(f'the busplan for bus {b} is not correct after row {i}, de battery status is then {bat_percentage:.2f}%')
                # overslaan naar volgende bus
                while i < n and df["bus"][i] == b:
                    i += 1
                break

            i += 1

    if incorrect == 0:
        print("the busplan werkt voor de batterijstatus")

In [50]:
batterij(df2, 300, 90, 85, 10)

the busplan for bus 1 is not correct after row 24, de battery status is then 7.84%
the busplan for bus 2 is not correct after row 87, de battery status is then 7.84%
the busplan for bus 3 is not correct after row 150, de battery status is then 6.88%
the busplan for bus 4 is not correct after row 205, de battery status is then 7.33%
the busplan for bus 5 is not correct after row 249, de battery status is then 7.68%
the busplan for bus 6 is not correct after row 288, de battery status is then 8.30%
the busplan for bus 7 is not correct after row 329, de battery status is then 8.27%
the busplan for bus 8 is not correct after row 370, de battery status is then 9.00%
the busplan for bus 9 is not correct after row 424, de battery status is then 8.30%
the busplan for bus 10 is not correct after row 470, de battery status is then 8.00%
the busplan for bus 11 is not correct after row 507, de battery status is then 8.10%
the busplan for bus 12 is not correct after row 540, de battery status is th